In [ ]:
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session()

batch_size = 100
epoch_num = 500
clip = [-0.1, 0.1]
lr = 0.0002
CRITIC_NUM = 5

In [ ]:
# 下載 dataset
import os
from urllib.request import urlretrieve
url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
tar_gz = '../cifar-10/cifar-10-python.tar.gz'
if not os.path.isfile(tar_gz):
    print('Downing data from %s' % url)
    urlretrieve(url, tar_gz)

In [ ]:
import tarfile
import pickle
import numpy as np
train_X = []
train_y = []
with tarfile.open(tar_gz) as tarf:
    for i in range(1, 6):
        dataset = "cifar-10-batches-py/data_batch_%d"%i
        print('load', dataset)
        with tarf.extractfile(dataset) as f:
            result = pickle.load(f, encoding='latin1')
            print(result['data'].shape)
            train_X.extend(np.moveaxis(result['data'].reshape(-1, 3, 32, 32) / 255 * 2 - 1, 1, 3))
            train_y.extend(result['labels'])
    train_X = np.float32(train_X)
    train_y = np.float32(train_y)
    dataset = 'cifar-10-batches-py/test_batch'
    with tarf.extractfile(dataset) as f:
        result = pickle.load(f, encoding='latin1')
        test_X = np.float32(np.moveaxis(result['data'].reshape(-1, 3, 32, 32) / 255 * 2 - 1, 1, 3))
        test_y = np.float32(result['labels'])
print(train_X.shape)
print(train_y.shape)
print(test_X.shape)
print(test_y.shape)

In [ ]:
import matplotlib.pyplot as plt
def showX(X, rows=1):
    length = X.shape[0]
    plt.figure(figsize=(length / rows, rows))
    print(X.shape)
    int_X = ((X+1)/2*255).clip(0, 255).astype('uint8')
    print(int_X.shape)
    for i in range(length):
        plt.subplot(rows, length / rows, i + 1)
        plt.imshow(int_X[i])
        plt.axis('off')
    plt.show()
showX(train_X[100:110], rows=2)

In [ ]:
def lrelu(x, leak=0.2):
    return tf.maximum(x, x*0.2)

In [ ]:
def discriminator(x, isTrain=True, reuse=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        conv1 = tf.layers.conv2d(x, 64, [4, 4], strides=(2, 2), padding='same')
        lrelu1 = lrelu(conv1)

        conv2 = tf.layers.conv2d(lrelu1, 128, [4, 4], strides=(2, 2), padding='same')
        lrelu2 = lrelu(tf.layers.batch_normalization(conv2, training=isTrain))

        conv3 = tf.layers.conv2d(lrelu2, 256, [4, 4], strides=(2, 2), padding='same')
        lrelu3 = lrelu(tf.layers.batch_normalization(conv3, training=isTrain))

        conv4 = tf.layers.conv2d(lrelu3, 1, [4, 4], strides=(1, 1), padding='valid')
        return conv4

In [ ]:
def generator(z, isTrain=True, reuse=False):
    with tf.variable_scope('generator', reuse=reuse):
        conv1 = tf.layers.conv2d_transpose(z, 256, [4, 4], strides=(1, 1), padding='valid')
        lrelu1 = tf.nn.relu(tf.layers.batch_normalization(conv1, training=isTrain))

        conv2 = tf.layers.conv2d_transpose(lrelu1, 128, [4, 4], strides=(2, 2), padding='same')
        lrelu2 = tf.nn.relu(tf.layers.batch_normalization(conv2, training=isTrain))

        conv3 = tf.layers.conv2d_transpose(lrelu2, 64, [4, 4], strides=(2, 2), padding='same')
        lrelu3 = tf.nn.relu(tf.layers.batch_normalization(conv3, training=isTrain))

        conv4 = tf.layers.conv2d_transpose(lrelu3, 3, [4, 4], strides=(2, 2), padding='same')
        out = tf.nn.tanh(conv4)
        return out

In [ ]:
X = tf.placeholder(dtype=tf.float32, shape=[None, 32, 32, 3])
Z = tf.placeholder(dtype=tf.float32, shape=[None, 1, 1, 100])
isTrain = tf.placeholder(dtype=tf.bool)
G_sample = generator(Z)

D_real = discriminator(X, isTrain)
D_fake = discriminator(G_sample, isTrain, True)

D_loss_fake = tf.reduce_mean(D_fake)
D_loss_real = tf.reduce_mean(D_real)
D_loss = D_loss_fake - D_loss_real
G_loss = -tf.reduce_mean(D_fake)

T_vars = tf.trainable_variables()
D_vars = [var for var in T_vars if var.name.startswith('discriminator')]
G_vars = [var for var in T_vars if var.name.startswith('generator')]

D_optimizer = tf.train.RMSPropOptimizer(lr).minimize(D_loss, var_list=D_vars)
G_optimizer = tf.train.RMSPropOptimizer(lr).minimize(G_loss, var_list=G_vars)
clip_D_op = [var.assign(tf.clip_by_value(var, clip[0], clip[1])) for var in D_vars]

sess.run(tf.global_variables_initializer())

In [ ]:
for epoch in range(epoch_num):
    for i in range(int(train_X.shape[0] / batch_size)):
        if i < 25:
            D_num = 25
        else:
            D_num = CRITIC_NUM
            
        for j in range(D_num):
            Z_ = np.random.uniform(-1, 1, size=(batch_size, 1, 1, 100))
            X_ = train_X[i*batch_size:(i+1)*batch_size]
            _, D_loss_curr = sess.run([D_optimizer, D_loss], feed_dict={X: X_, Z: Z_, isTrain: True})
            sess.run(clip_D_op)
        Z_ = np.random.uniform(-1, 1, size=(batch_size, 1, 1, 100))
        _, G_loss_curr = sess.run([G_optimizer, G_loss], feed_dict={Z: Z_, isTrain: True})
    print('epoch {}, D_loss_curr {}, G_loss_curr {}'.format(epoch, D_loss_curr, G_loss_curr))
    Z_demo = np.random.uniform(-1, 1, size=(20, 1, 1, 100))
    G_display = sess.run(G_sample, feed_dict={Z: Z_demo})
    showX(G_display, 4)